In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = (16, 9)

# Human Resources : who is going to quit ?

> <div class="alert alert-block alert-info">
The purpose of the notebook is to try to determine the factors that can alert a resignation.
</div>




In [ ]:
# load data
df = pd.read_csv('../input/human-resources-data-set/HRDataset_v13.csv')

## Data description
Description of fields is available here : https://rpubs.com/rhuebner/HRCodebook-13

* Employee Name : Employee’s full name
* EmpID : Employee ID is unique to each employee
* MarriedID : Is the person married (1 or 0 for yes or no)
* MaritalStatusID : Marital status code that matches the text field MaritalDesc
* EmpStatusID : Employment status code that matches text field EmploymentStatus
* DeptID : Department ID code that matches the department the employee works in
* PerfScoreID : Performance Score code that matches the employee’s most recent performance score
* FromDiversityJobFairID : Was the employee sourced from the Diversity job fair? 1 or 0 for yes or no
* PayRate : The person’s hourly pay rate. All salaries are converted to hourly pay rate
* Termd : Has this employee been terminated - 1 or 0
* PositionID : An integer indicating the person’s position
* Position : The text name/title of the position the person has
* State : The state that the person lives in
* Zip : The zip code for the employee
* DOB : Date of Birth for the employee
* Sex : Sex - M or F
* MaritalDesc : The marital status of the person (divorced, single, widowed, separated, etc)
* CitizenDesc : Label for whether the person is a Citizen or Eligible NonCitizen
* HispanicLatino : Yes or No field for whether the employee is Hispanic/Latino
* RaceDesc : Description/text of the race the person identifies with
* DateofHire : Date the person was hired
* DateofTermination : Date the person was terminated, only populated if, in fact, Termd = 1
* TermReason : A text reason / description for why the person was terminated
* EmploymentStatus : A description/category of the person’s employment status. Anyone currently working full time = Active
* Department : Name of the department that the person works in
* ManagerName : The name of the person’s immediate manager
* ManagerID : A unique identifier for each manager.
* RecruitmentSource : The name of the recruitment source where the employee was recruited from
* PerformanceScore : Performance Score text/category (Fully Meets, Partially Meets, PIP, Exceeds)
* EngagementSurvey : Results from the last engagement survey, managed by our external partner
* EmpSatisfaction : A basic satisfaction score between 1 and 5, as reported on a recent employee satisfaction survey
* SpecialProjectsCount : The number of special projects that the employee worked on during the last 6 months
* LastPerformanceReviewDate : The most recent date of the person’s last performance review.
* DaysLateLast30 : The number of times that the employee was late to work during the last 30 days

In [ ]:
df.dropna(how='all',inplace=True)
df.head()

In [ ]:
# Missing Values
missings = df.isnull().sum()
print(missings[missings>0])

## Preprocessing and feature engineering

In [ ]:
# Missing values
df['DaysLateLast30'] = df['DaysLateLast30'].fillna(-1)
# Conversion to datatime
df['DateofHire'] = pd.to_datetime(df['DateofHire'], dayfirst = False)
df['DOB'] = pd.to_datetime(df['DOB'], dayfirst = False)
df['DateofTermination'] = pd.to_datetime(df['DateofTermination'], dayfirst = False)
df['LastPerformanceReview_Date'] = pd.to_datetime(df['LastPerformanceReview_Date'], dayfirst = False)
 
#df['Last_Name'] = df['Employee_Name'].apply(lambda name: name.split(',')[0].strip())
#df['first_Name'] = df['Employee_Name'].apply(lambda name: name.split(',')[1].strip() if ("," in name) else "")

# Cleaning
df["Sex"] = df["Sex"].str.strip()
# bining PayRate
bins = [10,20,30,40,50,60,70,80,90]
labels = [10,20,30,40,50,60,70,80]
df["PayRateBin10"] = pd.cut(df["PayRate"], bins)

**New column : Number of days present in the company  **

* if terminated (DateofTermination - DateofHire)
* else (Today - DateofHire)

What is today ? 

I've taken the last performance review date 


In [ ]:
df['LastDayWorked'] = df['DateofTermination']
df['LastDayWorked'].fillna(df['LastPerformanceReview_Date'].max(),inplace=True)
df['NbrDayWorked'] = (df['LastDayWorked']-df['DateofHire'])//np.timedelta64(1,'D')
del df['LastDayWorked']
print(df['NbrDayWorked'].max(),df['NbrDayWorked'].min())

**New column : Gap to median of the pay rate in the position group**

In [ ]:
df["PayRateMedianPosition"] = df.groupby(['Position'])["PayRate"].transform('median')
df["PayRateEcart"] = df["PayRate"] - df["PayRateMedianPosition"]

## Normalization of the data


In [ ]:
# Columns to keep by nature
keepColId = ['MarriedID', 'FromDiversityJobFairID','Termd']
keepColNum = ['NbrDayWorked','PayRateEcart', 'EngagementSurvey', 'EmpSatisfaction','SpecialProjectsCount']
keepColCat = ['Sex', 'MaritalDesc', 'CitizenDesc','Department', 'PerformanceScore']

# Normalization
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaledData = scaler.fit_transform(df[keepColNum])
scaledData=pd.DataFrame(scaledData,columns=keepColNum)

# OneHotEncoding
data = pd.concat([scaledData,df[keepColId], pd.get_dummies(df[keepColCat]),df['EmploymentStatus']], axis=1)

# Features
X=data.loc[data['EmploymentStatus'].isin(['Active','Voluntarily Terminated'])].copy()
# Target
y=X['Termd']

X.drop(["Termd",'EmploymentStatus'], axis=1, inplace=True)

## Data exploration with a decision tree
> <div class="alert alert-block alert-info">
The purpose is to see the factors that have the most importance in a decision tree.<br/>
The target decision is Terminated or Not Terminated
</div>

The employment status are :
* Active = 1
* Future Start = 2
* Leave of Absence = 3 
* Terminated for Cause = 4
* Voluntarily Terminated = 5 

Termd column :
* Termd = 0 means No Terminated, i.e. [Active,Future Start,Leave of Absence]
* Termd = 1 means Terminated, i.e. [Terminated for Cause, Voluntarily Terminated]


In [ ]:
pd.crosstab(df['Termd'], df['EmploymentStatus'])

In [ ]:
pd.crosstab(df['EmpStatusID'], df['EmploymentStatus'])

In [ ]:
from sklearn import tree
import graphviz 

clf = tree.DecisionTreeClassifier()
clf.fit(X, y)

dot_data = tree.export_graphviz(clf, out_file=None, 
                     feature_names=X.columns,  
                     filled=True, rounded=True,  
                     special_characters=True)  
graph = graphviz.Source(dot_data)  
graph 

<div class="alert alert-block alert-info">
The features that have the most importances in the decision are :
</div>



In [ ]:
columns = X.columns
importances = list(zip(clf.feature_importances_,columns))
importances.sort(reverse=True)
pd.DataFrame(importances, index=[x for (_,x) in importances]).plot(kind = 'bar');

### let's see that with scatters plot

Termd column :
* Termd = 0 means No Terminated, i.e. [Active,Future Start,Leave of Absence]
* Termd = 1 means Terminated, i.e. [Terminated for Cause, Voluntarily Terminated]

In [ ]:
scatter = plt.scatter(X["NbrDayWorked"], X["SpecialProjectsCount"],c=y)
plt.xlabel("NbrDayWorked")
plt.ylabel("SpecialProjectsCount")
plt.legend(*scatter.legend_elements(),title="Classes")
plt.show()

scatter = plt.scatter(X["PayRateEcart"], X["MaritalDesc_Divorced"],c=y)
plt.xlabel("PayRateEcart")
plt.ylabel("MaritalDesc_Divorced")
plt.legend(*scatter.legend_elements(),title="Classes")
plt.show()

In [ ]:
pd.crosstab(X['Sex_F'], y)

In [ ]:
pd.crosstab(X['Department_Production       '], y)

In [ ]:
scatter = plt.scatter(df["NbrDayWorked"], df["PayRate"],c=df['EmpStatusID'])
plt.xlabel("NbrDayWorked")
plt.ylabel("PayRate")
plt.legend(handles=scatter.legend_elements()[0],labels=['Active','Future Start','Leave of Absence','Terminated for Cause','Voluntarily Terminated'])
plt.show()

<div class="alert alert-block alert-info">
Visually, the feature NbrDayWorked is really relevant,<br/>
<b>the longer the person stays in the company, the lower the probability of leaving</b>
</div>

## Data exploration with a logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression    
from sklearn.metrics import f1_score
from matplotlib.colors import ListedColormap, colorConverter, LinearSegmentedColormap

def visualize_coefficients(coefficients, feature_names, n_top_features=25):
    coefficients = coefficients.squeeze()
    if coefficients.ndim > 1:
        # this is not a row or column vector
        raise ValueError("coeffients must be 1d array or column vector, got"
                         " shape {}".format(coefficients.shape))
    coefficients = coefficients.ravel()

    if len(coefficients) != len(feature_names):
        raise ValueError("Number of coefficients {} doesn't match number of"
                         "feature names {}.".format(len(coefficients),
                                                    len(feature_names)))
    # get coefficients with large absolute values
    coef = coefficients.ravel()
    positive_coefficients = np.argsort(coef)[-n_top_features:]
    negative_coefficients = np.argsort(coef)[:n_top_features]
    interesting_coefficients = np.hstack([negative_coefficients,
                                          positive_coefficients])
    # plot them
    plt.figure(figsize=(20, 7))
    cm = ListedColormap(['#0000aa', '#ff2020'])
    colors = [cm(1) if c < 0 else cm(0)
              for c in coef[interesting_coefficients]]
    plt.bar(np.arange(2 * n_top_features), coef[interesting_coefficients],
            color=colors)
    feature_names = np.array(feature_names)
    plt.subplots_adjust(bottom=0.3)
    plt.xticks(np.arange(1, 1 + 2 * n_top_features),
               feature_names[interesting_coefficients], rotation=60,
               ha="right")
    plt.ylabel("Coefficient magnitude")
    plt.xlabel("Features")

In [ ]:
logreg = LogisticRegression()
logreg.fit(X, y)
y_predict = logreg.predict(X)
print("\n LogisticRegression f1_score: {:.5f}\n".format(f1_score(y, y_predict, average='weighted')))

In [ ]:
visualize_coefficients(logreg.coef_, X.columns,n_top_features=10)

<div class="alert alert-block alert-danger">
to be continued ...<br/>
<b>if you like my try, please upvote</b>
</div>